# 신 코로나 지역별 위험지표 제작 및 안전한 길찾기 플랫폼
## 주제
: 국민의 안전한 이동을 위해 **전국 코로나 위험도를 의미하는 위험지표**를 나타내고 위험지표 중 큰 비중을 차지하는 **유동 인구 데이터를 활용한 안전 경로 안내 서비스**   

## 목차  
### 0. 주제 선정 이유
### 1. 코로나 데이터 전처리 및 EDA 분석
    1. 라이브러리 호출  
    2. 데이터 시각화   
### 2. 교통 데이터 전처리
    1. 서울시 대중교통 이용량 데이터 시각화   
    2. TS교통카드 정류장 이용량 데이터 전처리   
    3. TS교통가드 정류장 데이터 전처리   
    4. 전처리한 데이터들을 정류장 ID로 통합
### 3. 대중교통 안전 경로 추천 프로세스 제작
    1. 배경  
    2. 구현 방안
### 4. 지역별 코로나 위험지표 회귀 모델 제작
    1. 길찾기 프로그램 제작
    2. 대중교통 Open API인 ODsay와 연동하여 Json 파일 받아오기   
### 5. AWS Server 구축 및 주피터 설치
### 6. 결론

#### 0. 주제 선정 이유  
* 수도권은 대중교통 이용자가 일 평균 약 800만명에 달한다. 대한민국 인구의 약 20%가 대중교통을 이용하는 셈이다. 이는 곧 국민 대다수가 코로나 위험에 노출되어 있다는 것을 의미한다. 영국 국민건강보험의 코로나19 가이드에 따르면 '근접 접촉'은 감염자 2m 이내에 15분 이상 머무르는 것을 의미한다. 따라서 과거 데이터를 통해 전철과 버스의 혼잡도를 미리 파악하여 혼잡도가 더 작은 경로를 제공하는 플랫폼을 제공함으로써 국민들의 감염확률을 줄이고자 한다. 또한 전국 코로나 감염에 위험한 지표가 될 수 있는 변수들을 찾아 회귀모델을 제작하여 종속변수가 높게 나오는 지역들을 구분하고 이를 통해 효과적인 방역에 보탬이 되고자 한다.

### 1. 코로나 데이터 전처리 및 EDA 분석
#### 1.1 라이브러리 호출

In [ ]:
import mapboxgl
import folium
import matplotlib.font_manager as fm
import seaborn as sns
import mapclassify
import json
import requests
import datetime as dt
import pandas as pd
import numpy as np
import geopandas as gpd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import plotly_express as px
import matplotlib as mpl
import matplotlib.font_manager as fm
import warnings
import os
import re
from random import *
from geopandas import GeoDataFrame
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
import statsmodels.formula.api as sm 
import random
import seaborn as sns
import matplotlib.pyplot as plt
from haversine import haversine
from sklearn.linear_model import LinearRegression # 선형회귀모델 생성 
from sklearn.model_selection import train_test_split # train/test set 생성 
from sklearn.metrics import mean_squared_error # MSE : 평균제곱오차 - model 평가 

warnings.filterwarnings(action='ignore')

In [ ]:
Region = pd.read.csv("Region.csv")
Gender = pd.read_csv("TimeGender.csv")
covid_case = pd.read_csv("Case.csv")
Patient = pd.read_csv("PatientInfo.csv")

Region.head()

일별 성별별 누적 확진자 및 사망자 추이
* 여성 확진자가 남성 확진자보다 더 많으나 사망자는 남자가 더 많다.

In [ ]:
# date 열을 데이터타임으로 변환 후 인덱스로 설정
Gender.date = pd.to_datetime(Gender.date)

Gender = Gender.set_index("date")

male = Gender.loc[Gender['sex']=='male', :]
female = Gender.loc[Gender['sex']=='female', :]

# 그래프 한글 환경 설정
%config InlineBackend.figure_format = 'retina'

import matplotlib.font_manager as fm
fontpath = 'NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()

In [ ]:
# 그래픽 크기 설정
plt.rcParams['font.size'] = 12
plt.rcParama['figure.figsize'] = (14, 8)

plt.plot(male['confirmed'], 'b', label='male')
plt.plot(female['confirmed'], 'r', label='female')
plt.grid()
plt.legend()
plt.title('일별 누적 확진자 추이', fontproperties=font)
plt.xlabel('Month')
plt.ylabel('누적 환자 수', fontproperties=font)
plt.show()

plt.plot(male['deceased'], 'b', label='male')
plt.plot(female['deceased'], 'r', label='female')
plt.grid()
plt.legend()
plt.title('일별 누적 사망자 추이', fontproperties=font)
plt.xlabel('Month')
plt.ylabel('누적 환자 수', fontproperties=font)
plt.show()

In [ ]:
# 그래픽 크기 설정
plt.rcParams["figure.figsize"] = (18, 10)

sum_tmp = covid_case.filter(['province', 'confirmed'])
province_sum = sum_tmp.groupby(['province']).sum()

# 지역별 확진자 발생현황 Bar 그래프 생성
province_sum['confirmed'].plot.bar(color='#AD8EDB', rot=0, width=0.5)
plt.grid()
plt.legend()
plt.title("지역별 확진자 발생현황",fontproperties=font)
plt.xlabel("지역",fontproperties=font)
plt.ylabel("확진자 수")
xpos = np.arange(len(province_sum.index))
plt.xticks(xpos,list(province_sum.index))

for x,y in enumerate(list(province_sum['confirmed'])):
    num = "%d명"%y
    plt.text(x, y, num, fontsize=11, color='#5D5D5D', horizontalalignment='center', verticalalignment='bottom')

plt.show()

* 지역별 집단 감염 발생현황 그래프

In [ ]:
# 그래픽 크기 설정
plt.rcParams["figure.figsize"] = (12, 5)

# 집단 감염이 많이 일어난 지역을 기준으로 데이터 분할
covid_group_tmp = covid_case[covid_case.group==True]
covid_group = covid_group_tmp[covid_group_tmp.confirmed>=30]
covid_group

# 집단 감염이 많이 일어난 지역 파악을 위한 그래프 출력 (한번에 30명 이상의 집단 감염이 있었던 사례)
group_sum_tmp = covid_group.filter(['province', 'confirmed'])
group_province_sum = group_sum_tmp.groupby(['province']).sum()

group_province_sum['confirmed'].plot.bar(color='#AD8EDB', rot=0, width=0.5)
plt.grid()
plt.legend()
plt.title("지역별 집단 감염 발생현황")
plt.xlabel("지역")
plt.ylabel("확진자 수")
xpos = np.arange(len(group_province_sum.index))
plt.xticks(xpos,list(group_province_sum.index))

for x,y in enumerate(list(group_province_sum['confirmed'])):
    num = "%d명"%y
    plt.text(x, y, num, fontsize=11, color='#5D5D5D', horizontalalignment='center', verticalalignment='bottom')

plt.show()

* 집단 감염 사례 : 2위~6위

In [ ]:
# 그래픽 크기 설정
plt.rcParams["figure.figsize"] = (15, 7)

# 집단 감염 사례 상위 6개 확인
groupcase = covid_group_tmp.filter(['infection_case', 'confirmed'])
groupcase_tmp = groupcase.groupby(['infection_case']).sum()

groupcase_top6 = groupcase_tmp.sort_values('confirmed',ascending=False).head(6)

# 신천지 사례는 이미 독보적이므로 그를 제외한 다섯 가지 사례에 대한 pie chart 생성
groupcase_top = groupcase_top6.iloc[1:7,:]

c_map = plt.get_cmap('Spectral')
col = [c_map(i) for i in np.linspace(0, 1, 5)]

plt.title("집단 감염 사례 2위 - 6위")
plt.pie(groupcase_top.confirmed, labels=list(groupcase_top.index), autopct='%1.1f%%', shadow=True, colors=col)
plt.show()

In [ ]:
# 시, 도 단위로 데이터 분할 저장

state_data = covid_case.filter(['province', 'confirmed'])
state_data = state_data.groupby(['province'], as_index = False).sum()
state_data

In [ ]:
# Time 데이터 읽은 뒤 달에 해당하는 column 추가
# data와 time을 합쳐서 date에 다시 저장한 후 datatime 형식으로 변환

covid_time = pd.read_csv("Time.csv")
covid_time['month'] = None
for i in range(len(covid_time)):
    covid_time['month'][i] = int(convid_time['date'][i].split('-')[1])
    
for i in range(len(covid_time)):
    covid_time['date'][i] = "{} {}:00:00".format(covid_time['date'][i], covid_time['time'][i])
    
covid_time['date'] = covid_time['date'].astype('datetime64[ns]')
covid_time

In [ ]:
# 확진자 신규 확진자와 신규 완치자 column 추가

covid_time['newconfirmed'] = None
covid_time['newconfirmed'][0] = covid_time['confirmed'][0]

covid_time['newreleased'] = None
covid_time['newreleased'][0] = covid_time['released'][0]

for i in range(1, len(covid_time)):
    covid_time['newconfirmed'][i] = covid_time['confirmed'][i] - covid_time['confirmed'][i-1]
    covid_time['newreleased'][i] = covid_time['released'][i] - covid_time['released'][i-1]

* 시간에 따른 신규 확진자-완치자 추이 (실질확진자) 그래프

In [ ]:
covid_time1 = covid_time.filter(['date', 'newconfirmed', 'newreleased'])
covid_time1.set_index('date', inplace=True)

covid_time1['newconfirmed'].plot()
covid_time1['newreleased'].plot()
plt.grid()
plt.legend()
plt.title('시간에 따른 신규 확진자-완치자 추이 : 실질 확진자', fontproperties=font)
plt.xlabel('날짜')
plt.ylabel('신규 확진자 및 신규 완치자 수')

* 위 그래프는 일별 신규 확진자와 신규 완치자 추이를 나타내는 그래프이다. 신규 확진자 수가 신규 완치자 수를 훌쩍 넘는 4월 초중순 정도까지의 구간은 코로나 확진세가 잘 잡히지 않던 정황을 보여주고, 그 이후는 코로나 확진세가 완화되었음을 보여준다.

* 사랑제일교회 광화문 시위의 문제점에 주목할 필요가 있다. 집회 참가자 중 대다수는 집회 이후 귀가 시 감염의 위험을 안은 채로 대중교통을 이용하였음을 가정했을 때, 대중교통을 통한 코로나 전파 위험을 최소화 하는 것이 코로나의 확산 위험을 줄이는 데 일조할 것으로 예측할 수 있다. 따라서 대중교통의 혼잡도를 사전에 확인하고 상대적으로 혼잡하지 않은 수단을 이용할 수 있도록 유도하는 것이 중요하다.

In [ ]:
Patient

Age=Patient['age'].value_counts()
AgeDF=pd.DataFrame(Age)

AgeDF_order = AgeDF.sort_index(ascending=True)

AgeDF_order['age']

AgeDF = AgeDF.reset_index()
AgeDF.columns = ['age', 'confirmed']
AgeDF['age']=AgeDF['age'].str[0:2]

AgeDF['age'][8] = 0
AgeDF['age'][10] = 100

AgeDF['age'] = pd.to_numeric(AgeDF['age'])

AgeDF = AgeDF.sort_values(by = ['age'])
AgeDF = AgeDF.set_index('age')

AgeDF.plot(kind='bar', title='연령별 확진자', rot=0, color='slateblue')
plt.xlabel('연령대')
plt.ylabel('확진자 수 (명)')
plt.rcParmas['figure.figsize'] = (10, 4)
plt.rcParmas['lines,linewidth'] = 2

* 경제활동인구가 많은 연령대에서 확지자 수가 높은 것을 볼 수 있다. 이는 곧 경제활동 인구가 활발한 연령대들이 상대적으로 위험에 노출되어 있는 것을 의미한다.

#### 광역시별 데이터와 시군구별 데이터 split

* Region.csv 파일에서 제주도는 시군으로 나뉘어져있지 않기 때문에 City DataFrame에도 추가한다.
* Region.csv 파일에 오타가 있어 직접 수정한다.

In [ ]:
Metropolitan = [dict(Region.loc[index:]) for index in Region.index if (Region.loc[index, 'province']==Region.loc[index, 'city'])]
Metropolitan = pd.DataFrame(Metropolitan)
# 마지막행 제거
Metropolitan = Metropolitan.iloc[:-1]
City = [dict(Region.loc[index:]) for index in Region.index if (Region.loc[index, 'province']!= Region.loc[index, 'city'])]
City = pd.DataFrame(City)
# 제주도 추가
City = City.append(Metropolitan.iloc[-1,:], ignore_index=True)

- 자가격리자 수 분포 확인

In [ ]:
plt.rcParams['font.size'] = 16

# 데이터 분포 확인 : 자가격리자
import seaborn as sns
nursing = City['nursing_home_count']
sns.kdeplot(nursing)
plt.title('자가격리자 수 분포')
plt.show()

- 변수 간 상관분석 실시

In [ ]:
# 위도, 경도 코드 column 삭제
City_cor = City.drop(['code', 'province', 'city', 'latitude', 'longitude'], axis=1)
plt.figure(figsize=(8, 8))
sns.heatmap(data=City_cor.corr(), annot=True,
            fmt='.2f', linewidths=.5, cmap=sns.diverging_palette(220, 20, as_cmap=True))

- 광역시별 자가격리자 시각화
: 원의 크기로 자가격리자 수 표현

In [ ]:
# 지도 초기화
Covid_map = folium.Map(location=[36, 127], tiles='OpenStreetMap', zoom_start=7)
for index in Metropolitan.index:
    lat = Metropolitan.loc[index, 'latitude']
    long = Metropolitan.loc[index, 'longitude']
    folium.CircleMarker([lat, long],
                        radius = Metropolitan.loc[index, 'nursing_home_count'/700],
                        popup = Metropolitan.loc[index, 'province'],
                        color = 'red',
                        fill = True.add_to(Covid_map))
Covid_map

* 시군구별 자가격리자 데이터 시각화

In [ ]:
Covid_map2 = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=7)
for index in City.index :
  lat = City.loc[index,'latitude']
  long = City.loc[index,'longitude']
  folium.CircleMarker([lat,long],
                     radius = City.loc[index,'nursing_home_count']/9,
                     popup = City.loc[index,'province'],
                     color = 'blue',
                     fill = True).add_to(Covid_map2)
Covid_map2

* shape file 불러오기 및 좌표계 변환   

In [ ]:
gdf = gpd.read_file('SIG.shp', encoding = 'EUC-KR')
gdf = gdf.to_crs({'init':'epsg:4326'})
gdf.plot();

* 외부 데이터와 내부 데이터 Merge (기준 : province, city)

In [ ]:
gdf1 = pd.read_excel('Revised_gps_data.xlsx')
gdf1.drop(['Unnamed: 0'],axis=1,inplace=True)
gdf.SIG_CD = gdf1['province']
gdf.SIG_KOR_NM = gdf1['city']
gdf = gdf.drop(['SIG_ENG_NM'], axis = 1)
gdf.columns = ['province','city','geometry']
City = pd.read_excel("Revised_City.xlsx")
Merge_data = pd.merge(left=gdf, right=City, how='left', on=['province','city'], sort=False)
Merge_data.drop(['Column1','code'],axis = 1, inplace = True)
Merge_data.head(1)

* 시군구별 자가격리자 수 데이터 시각화

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
Merge_data.plot(ax=ax, column="nursing_home_count", cmap="Reds", edgecolor="grey", linewidth=0.4, legend=True)
ax.axis("off")
plt.axis('equal')
plt.show()

위 지도를 보면 수도권에 자가격리자가 집중되어 잇는 것을 확인할 수 있다. 따라서 서울을 중점적으로 분석하기로 한다. 

** 교통 데이터 전처리 --> R

In [ ]:
# 지하철역 이용량 데이터
Subway = pd.read_csv('Subway_Popularity_July.csv', encoding='cp949')

# 버스정류장 이용량 데이터
Bus = pd.read_csv('BUS3_2.csv', encoding='cp949')

# 지하철역 위도 경도 데이터
SubwayLongLat = pd.read_csv('SubwayLocation.csv', encoding='utf-8')

# 버스정류장 위도 경도 데이터
BusLongLat = pd.read_csv('BusLocation.csv', encoding='cp949')

# 확진자 방문장소 데이터
Visitor = pd.read_csv('Place_Visitors.csv', encoding='utf-8')

데이터 전처리

In [ ]:
ColumnList = []
NameList = []


# column명 전처리
Subway.columns = Subway.columns.str.replace(" ","")
Subway.columns.values[:11] = Subway.columns[:11].str.replace("0","")
for Column in Subway.columns:
  ColumnList.append(Column.split('~')[0])
Subway.columns = ColumnList

# column명 변경 및 필요없는 column 삭제
Subway['역명'].str.split('(')[0]
Subway = Subway.rename({'6시이전':'5','24시이후':'24'}, axis='columns')
Subway = Subway.drop(['구분','할인','호선','역번호','합계'], axis='columns')

# 필요한 열 기준으로 그룹화하여 합산
Subway = Subway.groupby(['날짜','역명']).sum()
Subway = Subway.reset_index()

# 역명 뒤에 ()붙으면 그 앞까지만 저장
for Station in Subway['역명']:
  NameList.append(Station.split('(')[0])

Subway['역명'] = NameList

# 날짜형식 변환 및 요일 column 추가
Subway['날짜'] = pd.to_datetime(Subway['날짜'])
Subway.insert(1,'요일',Subway['날짜'].dt.dayofweek)

# 필요한 열 기준으로 그룹화하여 합산
Subway_DayOfWeek = Subway.groupby(['요일','역명']).sum()
Subway_DayOfWeek = Subway_DayOfWeek.reset_index()

Subway_DayOfWeek.head()

지하철 역별 위도 경도 좌표 데이터

In [ ]:
#불필요한 열 제거
SubwayLongLat = SubwayLongLat.drop(['Unnamed: 3','place'], axis='columns')

# Subway_DayOfWeek 데이터프레임에 맞게 문자열 처리, column 명 변환
SubwayLongLat['address'] = SubwayLongLat['address'].str[:-1]
SubwayLongLat['address'][SubwayLongLat['address'] == '서울'] = '서울역'
SubwayLongLat = SubwayLongLat.rename({'address' : '역명'}, axis='columns')
SubwayLongLat.head()

버스 정류장별 데이터

In [ ]:
# 날짜 형식으로 변환
Bus['DATE'] = Bus['DATE'].astype(str)
Bus['DATE'] = Bus['DATE'].str[:4] + '-' + Bus['DATE'].str[4:6] + '-' + Bus['DATE'].str[-2:]
Bus['DATE'] = pd.to_datetime(Bus['DATE'])

# 필요없는 column 제거 및 요일 column 추가
Bus.columns = ['시간', '날짜', '시군구코드','역번호','역명','승차인원','하차인원']
Bus.insert(1,'요일',Bus['날짜'].dt.dayofweek)
Bus = Bus.drop(['날짜','시군구코드','역번호','하차인원'], axis='columns')

# 필요한 열 기준으로 그룹화하여 합산
Bus_DayOfWeek = Bus.groupby(['시간','요일','역명']).sum()
Bus_DayOfWeek = Bus_DayOfWeek.reset_index()

# 필요없는 column 제거 및 column명 변환
BusLongLat = BusLongLat.drop(['표준ID','ARS-ID','비고'], axis='columns')
BusLongLat = BusLongLat.rename({'정류장명':'역명', 'X좌표':'Longitude', 'Y좌표':'Latitude'}, axis='columns')
BusLongLat.head()

#### 함수 구축

In [ ]:
def TimeRoute(JsonFile):
  RouteList = []
  
  # 경로마다 PathType, TotalTime 저장
  for PathType in JsonFile['result']['path']:
    Route = [{'PathType' : PathType['pathType'], 'TotalTime' : PathType['info']['totalTime']}]

    # trafficType 확인 후 해당 데이터셋에서 조회하여 필요한 값들 저장
    for Path in PathType['subPath'][1:-1]:
      if Path['trafficType'] == 1:
        Route.append({'TrafficType' : Path['trafficType'], 'StationID' : Path['startID'], 'Lane' : Path['lane'][0]['name'], 'StationName' : Path['startName']})

      elif Path['trafficType'] == 2:
        Route.append({'TrafficType' : Path['trafficType'], 'StationID' : Path['startID'], 'Lane' : Path['lane'][0]['busNo'], 'StationName' : Path['startName']})


    Route.append({'StationID' : Path['endID'], 'StationName' : Path['endName']})
    RouteList.append(Route)


  return(RouteList)

In [ ]:
def NumOfPeople(TimeRouteFile):
  DayOfWeek = dt.datetime.now().weekday()
  RouteTime = dt.datetime.now().hour
  RouteSumList = []
  TypeList = []

  # PathType과 TrafficType을 확인하여 적절한 데이터셋으로부터 유동인구를 저장
  for Route in TimeRouteFile:
    RouteSum = []
    Type = []
    if Route[0]['PathType'] == 1:
      for Station in Route[1:]:
        RouteSum.append(sum(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek][Subway_DayOfWeek['역명'] == Station['StationName']]))
        Type.append(1)

    elif Route[0]['PathType'] == 2:
      for Station in Route[1:]:
        RouteSum.append(sum(Bus_DayOfWeek['승차인원'][Bus_DayOfWeek['시간']==RouteTime][Bus_DayOfWeek['요일']==DayOfWeek][Bus_DayOfWeek['역명']==Station['StationName']]))
        Type.append(2)
    
    else:
      for Station in Route[1:-1]:
        if Station['TrafficType'] == 1:
          RouteSum.append(sum(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek][Subway_DayOfWeek['역명'] == Station['StationName']]))
          Type.append(1)
        else:
          RouteSum.append(sum(Bus_DayOfWeek['승차인원'][Bus_DayOfWeek['시간']==RouteTime][Bus_DayOfWeek['요일']==DayOfWeek][Bus_DayOfWeek['역명'] == Station['StationName']]))
          Type.append(2)
      if Station['TrafficType'] == 1:
        RouteSum.append(sum(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek][Subway_DayOfWeek['역명'] == str(Route[-1]['StationName'])]))
        Type.append(1)
      else:
        RouteSum.append(sum(Bus_DayOfWeek['승차인원'][Bus_DayOfWeek['시간']==RouteTime][Bus_DayOfWeek['요일']==DayOfWeek][Bus_DayOfWeek['역명']==Route[-1]['StationName']]))
        Type.append(2)
      
    TypeList.append(Type)
    RouteSumList.append(RouteSum)



  return(RouteSumList,TypeList)

In [ ]:
def RouteIndex(NumOfPeopleFile):
  NewNumList = []
  IndexList = []
  NewIndexList = []

  # 중복 경로 제거하는 반복문 생성
  for idx, NumList in enumerate(NumOfPeopleFile[0]):
    if NumList not in NewNumList:
      NewNumList.append(NumList)
      IndexList.append(idx)
      NewIndexList.append(NumOfPeopleFile[1][idx])

  NewNumList = NewNumList[:5]
  IndexList = IndexList[:5]
  NewIndexList = NewIndexList[:5]

  return(NewNumList, IndexList, NewIndexList)

In [ ]:
def CongestionLevel(RouteIndexFile):
  DayOfWeek = dt.datetime.now().weekday()
  RouteTime = dt.datetime.now().hour

  # 기초 level 단위 지정
  PopSubLevel = 1000
  PopBusLevel = 115

  # 최대 가능 level 계산
  MaxPopSub = np.max(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek])//PopSubLevel + 1
  MaxPopBus = np.max(Bus['승차인원'][Bus['시간'] == RouteTime][Bus['요일'] == DayOfWeek])//PopBusLevel + 1
  
  # Scaling Ratio 계산
  Ratio = MaxPopBus / MaxPopSub

  LevelList1 = []
  LevelList = []

  # 혼잡도 점수 계산
  for i in range(len(RouteIndexFile[1])):
    Level = []
    for j in range(len(RouteIndexFile[2][i])):
      if RouteIndexFile[2][i][j] == 1:
        Level.append((RouteIndexFile[0][i][j] // PopSubLevel + 1) * Ratio)
      
      else:
        Level.append(RouteIndexFile[0][i][j] // MaxPopBus + 1)
    
    LevelList1.append(round(sum(Level), 4))

  for Levels in LevelList1:
    LevelList.append(round(Levels/sum(LevelList1),4))
  
  return(LevelList)

In [ ]:
def TransferStation(TimeRouteFile):
  Idx = RouteIndex(NumOfPeople(TimeRouteFile))[1]
  TransferList = []

  # 환승지 데이터만 빼서 저장
  for idx in Idx:
    TransferStation = []
    for Station in TimeRouteFile[idx][1:-1]:
      TransferStation.append({'TrafficType' : Station['TrafficType'], 'StationName' : Station['StationName']})
    TransferList.append(TransferStation)
  
  return(TransferList)

In [ ]:
def LongLat(TransferStationFile):
  LongLatList = []
  for Stations in TransferStationFile:
    LongLat = []

    # TrafficType 확인 후 적절한 데이터 셋에서 위도와 경도 좌표 조회하여 저장
    for Station in Stations:
      if Station['TrafficType'] == 1:
        LongLat.append({'Latitude' : np.max(SubwayLongLat['Latitude'][SubwayLongLat['역명']==Station['StationName']].astype('float')), 'Longitude' : np.max(SubwayLongLat['Longitude'][SubwayLongLat['역명']==Station['StationName']].astype('float'))})
      else:
        LongLat.append({'Latitude' : np.max(BusLongLat['Latitude'][BusLongLat['역명']==Station['StationName']].astype('float')), 'Longitude' : np.max(BusLongLat['Longitude'][BusLongLat['역명']==Station['StationName']].astype('float'))})
    
    LongLatList.append(LongLat)
  
  return(LongLatList)

In [ ]:
def DangerLevel(LongLatFile):
  CountList = []
  CountList1 = []
  for Long_Lats in LongLatFile:
    CountList2 = []
    for Long_Lat in Long_Lats:
      Count = 0

      # haversine 모듈 이용하여 거리 계산 및 count
      for i in range(len(Visitor)):
        if haversine((Long_Lat['Latitude'], Long_Lat['Longitude']), (Visitor['Latitude'][i],Visitor['Longitude'][i]), unit = 'km') < 3:
          Count += 1
      CountList2.append(Count)
    CountList1.append(sum(CountList2))
  
  for Counts in CountList1:
    CountList.append(round(Counts/sum(CountList1)*0.4,4))


  return(CountList)

In [ ]:
def Recommendation(TimeRouteFile, RouteIndexFile, CongestionLevelFile, DangerLevelFile):
  TotalPoint = []
  RouteList = []

  # 위험도 지표 2개 합산하여 총점 산출
  for i in range(len(CongestionLevelFile)):
    TotalPoint.append(CongestionLevelFile[i] + DangerLevelFile[i])
  
  # 총점 중 최소값 찾기 (가장 안전한 경로 찾기)
  for idx, point in enumerate(TotalPoint):
    if point == min(TotalPoint):
      Index = idx
  
  # 경로 저장
  for i in range(len(RouteIndexFile[1])):
    Route = []
    for StationInfo in TimeRouteFile[i][1:-1]:
      Route.append('({}){}'.format(StationInfo['Lane'], StationInfo['StationName']))
    Route.append('{}'.format(TimeRouteFile[i][-1]['StationName']))
    RouteList.append(Route)

  BestRoute = RouteList[Index]

  return(RouteList,BestRoute,Index)

#### Json 파일 불러오기 및 함수 적용

In [ ]:
a = TimeRoute(TransportJson)

# 상위 3개만 출력
a[:3]

In [ ]:
b = NumOfPeople(a)

b

In [ ]:
c = RouteIndex(b)

c

In [ ]:
d = CongestionLevel(c)

d

In [ ]:
e = TransferStation(a)

e

In [ ]:
f = LongLat(e)

f

In [ ]:
g = DangerLevel(f)

g

In [ ]:
h = Recommendation(a,c,d,g)

Route = h[0]
BestRoute = h[1]
BestIndex = h[2]

h

최종 결과 출력

In [ ]:
print("""목적지까지의 경로입니다.

경로 1 : {} / {}분 소요
경로 2 : {} / {}분 소요
경로 3 : {} / {}분 소요
경로 4 : {} / {}분 소요
경로 5 : {} / {}분 소요


안전 지표 기반 최적 경로 :
***********************************************************************
{} / {}분 소요
***********************************************************************


(위 결과는 경로 상의 출발지, 환승지, 도착지의 혼잡도 지표와 환승지 근처의 확진자 방문장소 수 지표를 기반으로 추천된 경로입니다.)
""".format(' -> '.join(Route[0]), a[c[1][0]][0]['TotalTime'],
           ' -> '.join(Route[1]), a[c[1][1]][0]['TotalTime'],
           ' -> '.join(Route[2]), a[c[1][2]][0]['TotalTime'],
           ' -> '.join(Route[3]), a[c[1][3]][0]['TotalTime'],
           ' -> '.join(Route[4]), a[c[1][4]][0]['TotalTime'],
           ' -> '.join(BestRoute), a[c[1][BestIndex]][0]['TotalTime']))